In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [8]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [9]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [10]:
Poly_Ring.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i_rad3*c})

NameError: name 'i_rad3' is not defined

In [6]:
# Find simmetries

In [7]:
%time adm_SE9 = SE9.find_admissible_projectivities()
len(adm_SE9)

NameError: name 'SE9' is not defined

NameError: name 'adm_SE9' is not defined

In [ ]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)
len(simm_SE9)

In [ ]:
# Study how the simmetries permute the Eckardt points

In [ ]:
Eck_points_perms = []
for proj in simm_SE9:
    Eck_points_perms.append(Permutation(SE9.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

In [ ]:
# Study how the simmetries permute the 27 lines

In [ ]:
lines_perms = []
for simm in simm_SE9:
    lines_perms.append(Permutation(SE9.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

In [ ]:
all_subgroups = lines_perms_group.subgroups()

In [ ]:
# H is the subgroup of elements of order 3
H = [el for el in lines_perms_group.normal_subgroups() if el.order()==27][0]
list(set([el.order() for el in H if el.cycle_tuples() != []])) == [3]

In [ ]:
# The center is C3
Z_H = H.center()
M = Z_H.gens_small()[0]
Z_H.structure_description()

In [ ]:
H_over_Z_H = [el for el in all_subgroups if el.is_isomorphic(H.quotient(Z_H))][0]
[N, P] = H_over_Z_H.gens_small()
H_over_Z_H.structure_description()

In [ ]:
H.quotient(PermutationGroup([M, N])).is_isomorphic(PermutationGroup([P]))

In [ ]:
C2 = [el for el in all_subgroups if el.is_isomorphic(lines_perms_group.quotient(H))][0]
A = C2.gens_small()[0]

In [ ]:
gens = [M, N, P, A]
gens_indices = [lines_perms.index(gen) for gen in gens]
matrix_gens = [simm_SE9[index] for index in gens_indices]

In [ ]:
# There are twelve lines through 3 Eckardt points, and they are all coplanar

In [ ]:
lines_three_eck = []
lines_labels = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9.eckardt_points[i]
            point2 = SE9.eckardt_points[j]
            point3 = SE9.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                lines_labels.append([i+1,j+1,k+1])
                planes = get_two_planes_containing_line(point1, point2)
                lines_three_eck.append(Line(planes, [point1, point2], line12_pl))
print(lines_labels)

In [ ]:
bool([[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck])

In [ ]:
lines_three_eck[0].get_plane_containing_another_incident_line(lines_three_eck[1])

In [ ]:
# To do, study how the 12 lines are permuted

In [ ]:
for gen in matrix_gens:
    print([lines_three_eck.index(line.apply_proj(gen)) for line in lines_three_eck])

In [ ]:
[Eck_points_perms[gen_index] for gen_index in gens_indices]

In [ ]:
[Eck_points_perms[lines_perms.index(el)] for el in H_over_Z_H]

In [ ]:
keys = list(SE9.cl_lines.keys())
print(SE9.eckardt_points_labels)
print()
for gen in gens:
    labels = from_perm_to_labels(gen)
    print([[keys[labels.index(label)] for label in Eck_label] for Eck_label in SE9.eckardt_points_labels])
    print()